# Voice Data Detects Disease

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#wfdb is the package that can read WFDB format file
import wfdb

In [ ]:
import os
from typing import Dict


def parse_info_txt(file_path: str) -> Dict:
    """
    Parse the -info.txt file to extract subject information.
    
    Parameters:
    - file_path (str): The path to the -info.txt file.
    
    Returns:
    - Dictionary with parsed information.
    """
    info_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            # Some lines contain multiple key-value pairs separated by a space
            # Each key-value is separated by a colon
            # We'll split the line into key-value pairs and then process each one
            pairs = line.strip().split('  ')
            for pair in pairs:
                if ': ' in pair:
                    key, value = pair.split(': ')
                    # Replace 'NU' with numpy's NaN to indicate missing values
                    if value == 'NU':
                        value = np.nan
                    else:
                        # Try to convert numerical values to float for consistency
                        try:
                            value = float(value.replace(',', '.'))
                        except ValueError:
                            pass
                    info_dict[key] = value
    return info_dict